In [1]:
import os
import onnx
import torch
from torch import nn
import tensorflow as tf
from onnx_tf.backend import prepare
from torchvision.models import resnet18,mobilenet_v2,ResNet18_Weights
import torchvision
import torchvision.transforms as transforms
import warnings
warnings.filterwarnings("ignore")


data_dir = os.path.abspath("../../data")
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), transforms.RandomErasing()])
trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

dataiter = iter(trainloader) 
images, labels = next(dataiter)

model = resnet18(weights = ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(in_features=512,out_features=10)


onnx_file = "../../results/coral/Coral_Resnet18_model.onnx"
tf_model_path = "../../results/coral/Coral_Resnet18_model.tf"


torch.onnx.export(
    model,
    images,
    onnx_file,
    opset_version=12,
    do_constant_folding=True,
    input_names=['input_1'],
    output_names=['output'],
)

onnx_model =  onnx.load(onnx_file)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

# tf_model = tf.keras.models.Sequential([
#     tf.keras.layers.InputLayer(input_shape=(3, 32, 32)),
#     tf.keras.layers.Lambda(lambda x: x.permute(0, 2, 3, 1)),
#     tf.keras.layers.Lambda(lambda x: model(torch.Tensor(x.numpy()).permute(0, 3, 1, 2)).detach().numpy())
# ])

2023-06-14 10:09:55.166772: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 10:09:55.702035: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Files already downloaded and verified
============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



2023-06-14 10:09:57.907039: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-14 10:09:57.907216: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-14 10:09:57.907938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

INFO:tensorflow:Assets written to: ../../results/coral/Coral_Resnet18_model.tf/assets


INFO:tensorflow:Assets written to: ../../results/coral/Coral_Resnet18_model.tf/assets


In [2]:
tflite_model_path = "../../results/coral/Coral_Resnet18_model.tflite"

IMAGE_SIZE = 32
(train_images, train_labels), (_, _) = tf.keras.datasets.cifar10.load_data()
def representative_data_gen():
    for i in range(100):
        # Get a random image from the CIFAR-10 training dataset
        image = train_images[i % len(train_images)]
        # Resize the image to the desired size
        image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
        # Normalize the pixel values
        image = tf.cast(image / 255., tf.float32)
        # Add an extra dimension to represent a batch of size 1
        image = tf.expand_dims(image, 0)
        # Yield the image as representative data
        yield [image]



converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
#converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8


tflite_model = converter.convert()
with open(tflite_model_path, 'wb') as f: 
    f.write(tflite_model)

170498071/170498071 [==============================] - 8s 0us/step


2023-06-14 10:10:13.046681: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-06-14 10:10:13.046711: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-06-14 10:10:13.047497: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ../../results/coral/Coral_Resnet18_model.tf
2023-06-14 10:10:13.054666: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-14 10:10:13.054680: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: ../../results/coral/Coral_Resnet18_model.tf
2023-06-14 10:10:13.083099: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-06-14 10:10:13.083687: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-06-14 10:10:13.154872: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op 

RuntimeError: tensorflow/lite/kernels/conv.cc:351 input_channel % filter_input_channel != 0 (2 != 0)Node number 2 (CONV_2D) failed to prepare.